# From Generative AI Solutions with Databricks
This code is designed to run on Databricks, not locally. Here's how you'd use it:

1. Log the model (run once in a Databricks notebook)

In [ ]:
import mlflow
from mlflow.models.signature import infer_signature
import pandas as pd
CATALOG = "demo"
SCHEMA  = "finance"
TABLE   = "sales_transactions"

sample_input = pd.DataFrame({
    "table_name": [TABLE],
    "catalog": [CATALOG],
    "schema": [SCHEMA]
})
sample_output = pd.DataFrame({
    "rules": ['[{"column": "amount", "rule": "amount > 0"}]']
})
signature = infer_signature(sample_input, sample_output)

with mlflow.start_run() as run:
    mlflow.pyfunc.log_model(
        artifact_path="data_quality_agent",
        python_model="python_model.py",
        code_paths=["code/data_quality_agent"],
        artifacts={
            "system_prompt": "artifacts/system_prompt.txt",
            "example_rules": "artifacts/example_rules.json"
        },
        signature=signature,
        pip_requirements="requirements.txt"
    )

run_id = run.info.run_id
print(f"Model logged with run ID: {run_id}")
model_uri = f"runs:/{run_id}/data_quality_agent"
print(f"Model logged with URI: {model_uri}")
print(f"You will need run ID in notebook 02. run_id = {run_id}")


2. Load and call it (in a separate notebook or job)

In [ ]:
import mlflow
import pandas as pd
model = mlflow.pyfunc.load_model(f"runs:/{run_id}/data_quality_agent")
result = model.predict(pd.DataFrame({
    "table_name": ["sales_transactions"],
    "catalog": ["prod_catalog"],
    "schema": ["finance"]
}))
print(result["rules"][0])
